In [77]:
!pip install z3-solver

In [78]:
from itertools import combinations
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one(bool_vars):
    return at_most_one_np(bool_vars) + [at_least_one(bool_vars)]

def at_least_k_np(bool_vars, k, name = ""):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_np(bool_vars, k, name = ""):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k, name = ""):
    return And(at_most_k_np(bool_vars, k, name), at_least_k_np(bool_vars, k, name))


In [84]:
from z3 import *
from math import log2
import numpy as np

# Parameters
n_couriers = 3
n_items = 5
max_load = [10, 15, 20]
size_item = [2, 3, 5, 7, 9]
all_distances = [[0, 2, 3, 5, 7, 9], [2, 0, 1, 4, 6, 8], [3, 1, 0, 3, 5, 7], [5, 4, 3, 0, 2, 4], [7, 6, 5, 2, 0, 2], [9, 8, 7, 4, 2, 0]]

"""n_couriers = 3
n_items = 7
max_load = [10, 15, 7]
size_item = [3, 2, 6, 8, 5, 4, 4]
all_distances = [[0, 3, 3, 6, 5, 6, 6, 2], [3, 0, 4, 3, 4, 7, 7, 3], [3, 4, 0, 7, 6, 3, 5, 3], [6, 3, 7, 0, 3, 6, 6, 4], [5, 4, 6, 3, 0, 3, 3, 3], [6, 7, 3, 6, 3, 0, 2, 4], [6, 7, 5, 6, 3, 2, 0, 4], [2, 3, 3, 4, 3, 4, 4, 0]]
"""

"""
- n = is the number of clientes                                      | --> n_items
- N =  is set of clients, with N ={1, ..., n}                        | --> Mi posso creare il set of items (per euguagliare il set of clieents)
- V is set of vetices (or nodes) with V = {0} u N                    |
- A  is set of arcs, with A = {(i, j) ∈ V^2  : i!= j}                |
- c_i,j is cost of travel over arc (i, j) ∈ A                        |
- Q is the vehicle capacity                                          |
- q_i is the amount that has to be delivered to customer i ∈ N       | --> size_item (it's a dictionary on yt tutorial)


set of customer C --> size_item
set of routes R --> x
vehicle capacity Q -> max_load
cost of route c_r --> ???
Customer visited on route r C(r) --> ???
max number of vehicles k --> n_couriers


contraints:

# Each node visited only once [y]
# Each courier depart and go back from depot [y]
# Each courier can go back and depart from depot at maximum one time [y]
# Constraint on the load of each courier [y]

# Each item must be delivered

"""

s = Optimize()



x = [[[Bool(f"x_{i}_{j}_{k}") for k in range(n_couriers)] for j in range(n_items+1)] for i in range(n_items+1)] # x[k][i][j] == True : route (i->j) is used by courier k | set of Archs
v = [[Bool(f"v_{i}_{k}") for k in range(n_couriers)] for i in range(n_items)] #vehicle k is assigned to node i



print(x)
print(v)
# - - - - - - - - - - - - - - - - - - - - - - - -  CONSTRAINTS - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

#implicit constraint: no routes from any node to itself
for k in range(n_couriers):
  s.add([x[i][i][k] == False for i in range(n_items+1)])

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

"""#for each vehicle if x[k][i][j] = True, (i, j) -> (j, l)
for k in range(n_couriers):
  s.add([Implies(x[i][j][k], x[j][l][k]) for i in range(n_items+1) for j in range(n_items+1) for l in range(1, n_items+1)])"""

#for each vehicle if x[k][i][j] = True, (i, j) -> (j, l)
for k in range(n_couriers):
    for i in range(n_items+1):
        for j in range(n_items+1):
            s.add(Implies(x[i][j][k], Or([x[j][l][k] for l in range(n_items+1) if l != j])))

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #


# To ensure that each node (i, j) is visited only once, for each node there is exactly one arc entering and leaving from it

for i in range(1, n_items+1):  # start from 1 to exclude the depot
  s.add(PbEq([(x[i][j][k], 1) for j in range(n_items+1) for k in range(n_couriers)], 1))  # each node is left exactly once by each courier

for j in range(1, n_items+1):  # start from 1 to exclude the depot
    s.add(PbEq([(x[i][j][k], 1) for i in range(n_items+1) for k in range(n_couriers)], 1))  # each node is entered exactly once by each courier

# each courier return from the depot
for k in range(n_couriers):
    s.add(PbEq([(x[i][0][k], 1) for i in range(n_items+1)], 1))

# each courier depart from the depot
for k in range(n_couriers):
    s.add(PbEq([(x[0][j][k], 1) for j in range(n_items+1)], 1))


#for each vehicle, the total load over its route must be smaller than its max load size
for k in range(n_couriers):
  s.add(PbLe([(v[i][k], size_item[i]) for i in range(n_items)], max_load[k]))

total_distance = Sum([If(x[i][j][k], all_distances[i][j], 0) for k in range(n_couriers) for i in range(n_items+1) for j in range(n_items+1)])
s.minimize(total_distance)


[[[x_0_0_0, x_0_0_1, x_0_0_2], [x_0_1_0, x_0_1_1, x_0_1_2], [x_0_2_0, x_0_2_1, x_0_2_2], [x_0_3_0, x_0_3_1, x_0_3_2], [x_0_4_0, x_0_4_1, x_0_4_2], [x_0_5_0, x_0_5_1, x_0_5_2]], [[x_1_0_0, x_1_0_1, x_1_0_2], [x_1_1_0, x_1_1_1, x_1_1_2], [x_1_2_0, x_1_2_1, x_1_2_2], [x_1_3_0, x_1_3_1, x_1_3_2], [x_1_4_0, x_1_4_1, x_1_4_2], [x_1_5_0, x_1_5_1, x_1_5_2]], [[x_2_0_0, x_2_0_1, x_2_0_2], [x_2_1_0, x_2_1_1, x_2_1_2], [x_2_2_0, x_2_2_1, x_2_2_2], [x_2_3_0, x_2_3_1, x_2_3_2], [x_2_4_0, x_2_4_1, x_2_4_2], [x_2_5_0, x_2_5_1, x_2_5_2]], [[x_3_0_0, x_3_0_1, x_3_0_2], [x_3_1_0, x_3_1_1, x_3_1_2], [x_3_2_0, x_3_2_1, x_3_2_2], [x_3_3_0, x_3_3_1, x_3_3_2], [x_3_4_0, x_3_4_1, x_3_4_2], [x_3_5_0, x_3_5_1, x_3_5_2]], [[x_4_0_0, x_4_0_1, x_4_0_2], [x_4_1_0, x_4_1_1, x_4_1_2], [x_4_2_0, x_4_2_1, x_4_2_2], [x_4_3_0, x_4_3_1, x_4_3_2], [x_4_4_0, x_4_4_1, x_4_4_2], [x_4_5_0, x_4_5_1, x_4_5_2]], [[x_5_0_0, x_5_0_1, x_5_0_2], [x_5_1_0, x_5_1_1, x_5_1_2], [x_5_2_0, x_5_2_1, x_5_2_2], [x_5_3_0, x_5_3_1, x_5_3_2], [x

In [85]:
r_temp = 0
if s.check() == sat:
  model = s.model()
  r = [[[ model.evaluate(x[i][j][k]) for k in range(n_couriers)] for j in range(n_items+1)] for i in range(n_items+1)]
  ut = [[model.evaluate(v[i][k]) for k in range(n_couriers)] for i in range(n_items)]

  print(r)
  print("\n")

  for k in range(n_couriers):
    for i in range(n_items+1):
      for j in range(n_items+1):
        if model.evaluate(x[i][j][k]) == True:
            print(x[i][j][k])
    print("\n")


  print(model.evaluate(total_distance))
  print("\n")


  # Objective: Minimize total distance traveled


else:
  print("UNSATISFIABLE")



[[[False, False, False], [True, False, False], [False, True, False], [False, False, True], [False, False, False], [False, False, False]], [[True, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False]], [[False, True, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False]], [[False, False, True], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False]], [[False, False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False], [True, False, False]], [[False, False, False], [False, False, False], [False, False, False], [False, False, False], [True, False, False], [False, False, False]]]


x_0_1_0
x_1_0_0
x_4_5_0
x_5_4_0


x_0_2_1
x_2_0_1


x_0_3_2
x_3_0_2


24




In [ ]:
"""
PbEq() function from Z3, which stands for "pseudo-boolean equality".
  This function takes two arguments: a list of pairs, where each pair consists of a Boolean variable and a weight, and a number.
  It returns True if the sum of the weights of the True variables is equal to the number.

PbLe() function from Z3, which stands for "pseudo-boolean less than or equal to".
  It takes two arguments: a list of pairs and a number. Each pair in the list consists of a boolean variable and an integer weight.
  The function returns True if the sum of the weights of the True variables is less than or equal to the number.




"""